# Screener Demo


In [13]:
import os
import requests
from typing import Dict, List, Any

# Base Screener Abstract Class
class BaseScreener:
    def get_available_criteria(self) -> Dict[str, Any]:
        pass
    
    def screen(self, criteria: Dict[str, Any]) -> List[Dict[str, Any]]:
        pass
    
    def get_stock_details(self, ticker: str) -> Dict[str, Any]:
        pass

# EODHD Screener Implementation
class EODHDScreener(BaseScreener):
    """
    Concrete BaseScreener using EODHD's Financial Data APIs.
    Quick-start docs: <https://eodhd.com/financial-apis/quick-start-with-our-financial-data-apis>[1]
    """

    def __init__(self, api_token: str = None):
        self.api_token = api_token or os.getenv("EODHD_API_TOKEN")
        if not self.api_token:
            raise ValueError("Set EODHD_API_TOKEN env var or pass api_token")
        self.base = "https://eodhd.com/api/"

    def _get(self, path: str, params: Dict[str, Any] = None) -> Any:
        url = f"{self.base}{path}"
        p = {"api_token": self.api_token, "fmt": "json"}
        if params:
            p.update(params)
        resp = requests.get(url, params=p, timeout=10)
        resp.raise_for_status()
        return resp.json()

    def get_available_criteria(self) -> Dict[str, Any]:
        """
        Hard-coded schema of supported filters.
        """
        return {
            "exchange":         ["NSE", "BSE", "US"],
            "market_cap_min":   1e8,        # in local currency
            "market_cap_max":   1e12,
            "pe_ratio_min":     0.0,
            "pe_ratio_max":     100.0,
            "div_yield_min":    0.0,
            "div_yield_max":    0.1,
            "price_min":        1.0,
            "price_max":        1e4,
            "limit":            100
        }

    def screen(self, criteria: Dict[str, Any]) -> List[Dict[str, Any]]:
        """
        1. Load all symbols for the given exchange.
        2. Fetch fundamentals for each.
        3. Apply client-side filters on market cap, PE, dividend yield, and latest price.
        """
        exch = criteria.get("exchange", "NSE")
        # 1. list symbols on that exchange
        symbols = self._get(f"exchange-symbol-list/{exch}", {})  # [ { "Code": "TCS.NS", ... }, ... ] [1]

        results = []
        limit = criteria.get("limit", 100)
        for sym in symbols:
            if len(results) >= limit:
                break
            code = sym.get("Code")
            data = self._get(f"fundamentals/{code}")  # fundamentals + EOD history [1]
            gen = data.get("General", {})
            hist = data.get("EOD", [])
            latest = hist[-1] if hist else {}
            
            # extract fields
            mc   = float(gen.get("MarketCapitalization") or 0)
            pe   = float(gen.get("PERatio") or 0)
            dy   = float(gen.get("DividendYield") or 0)
            price= float(latest.get("close") or 0)

            # apply filters
            if not (criteria["market_cap_min"] <= mc <= criteria["market_cap_max"]):
                continue
            if not (criteria["pe_ratio_min"] <= pe <= criteria["pe_ratio_max"]):
                continue
            if not (criteria["div_yield_min"] <= dy <= criteria["div_yield_max"]):
                continue
            if not (criteria["price_min"] <= price <= criteria["price_max"]):
                continue

            results.append({
                "ticker": code,
                "name": gen.get("Name"),
                "marketCap": mc,
                "peRatio": pe,
                "dividendYield": dy,
                "price": price
            })

        return results

    def get_stock_details(self, ticker: str) -> Dict[str, Any]:
        """
        Returns a unified detail structure combining fundamentals + latest EOD price.
        """
        data = self._get(f"fundamentals/{ticker}")  # [1]
        gen = data.get("General", {})
        hist = data.get("EOD", [])
        latest = hist[-1] if hist else {}

        return {
            "ticker":        gen.get("Symbol"),
            "name":          gen.get("Name"),
            "sector":        gen.get("Sector"),
            "industry":      gen.get("Industry"),
            "country":       gen.get("Country"),
            "marketCap":     gen.get("MarketCapitalization"),
            "peRatio":       gen.get("PERatio"),
            "dividendYield": gen.get("DividendYield"),
            "latestPrice":   latest.get("close")
        }

# Replace with your actual API token
screener = EODHDScreener("682793d3735129.70503041")
print("Filters:", screener.get_available_criteria())

# Set up screening criteria
crit = {
    "exchange":       "NSE",
    "market_cap_min": 1e10,
    "market_cap_max": 5e11,
    "pe_ratio_min":   0,
    "pe_ratio_max":   50,
    "div_yield_min":  0,
    "div_yield_max":  0.05,
    "price_min":      10,
    "price_max":      2000,
    "limit":          20
}

# Run the screen
hits = screener.screen(crit)
print(f"Found {len(hits)} stocks:", [h["ticker"] for h in hits])

# Get details for a specific stock
detail = screener.get_stock_details("TCS.NS")
print("TCS Details:", detail)

Filters: {'exchange': ['NSE', 'BSE', 'US'], 'market_cap_min': 100000000.0, 'market_cap_max': 1000000000000.0, 'pe_ratio_min': 0.0, 'pe_ratio_max': 100.0, 'div_yield_min': 0.0, 'div_yield_max': 0.1, 'price_min': 1.0, 'price_max': 10000.0, 'limit': 100}


HTTPError: 403 Client Error: Forbidden for url: https://eodhd.com/api/fundamentals/0P00009J3K?api_token=682793d3735129.70503041&fmt=json

In [11]:
!https://eodhd.com/api/exchange-symbol-list/USA?api_token=682793d3735129.70503041&fmt=json

zsh:1: no matches found: https://eodhd.com/api/exchange-symbol-list/USA?api_token=682793d3735129.70503041


# Alpha Vantage API Demo

This notebook demonstrates how to use the Alpha Vantage client for fetching financial data, fundamentals, and news & sentiment analysis.

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

# Add the project root to the path to enable imports
sys.path.append(os.path.abspath('../..'))

# Import the Alpha Vantage client
from investment_machine.src.tools.data_fetchers.alpha_vantage import AlphaVantageClient

/Users/paggrawal/hobby_projects/trader/investment_machine/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
!pip3 install -q pandas matplotlib

You should consider upgrading via the '/Users/paggrawal/hobby_projects/trader/investment_machine/venv/bin/python3 -m pip install --upgrade pip' command.


## Initialize the Alpha Vantage Client

You can provide your API key directly or set it as an environment variable `ALPHA_VANTAGE_API_KEY`.

In [3]:
# Option 1: Read from environment variable
# client = AlphaVantageClient()

# Option 2: Provide API key directly
client = AlphaVantageClient(api_key="JTPKOHNIT0NJYEQ2")

In [7]:
import json
data = client._make_request({
    "function": "TIME_SERIES_DAILY_ADJUSTED",
    "symbol": "IBM",
    "outputsize": "compact"
})

In [8]:
data

{'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}

In [9]:
import os
import json
import pandas as pd

# Log to store responses and errors
log = []

def safe_call(name, fn, *args, **kwargs):
    """Helper to call fn and print summary or error."""
    print(f"\n--- {name} ---")
    entry = {"name": name}
    try:
        res = fn(*args, **kwargs)
        entry["status"] = "success"

        # DataFrame?
        if isinstance(res, pd.DataFrame):
            entry["type"] = "DataFrame"
            entry["shape"] = res.shape
            entry["preview"] = res.head().to_dict()
            print("DataFrame shape:", res.shape)
            print(res.head())

        # Dict?
        elif isinstance(res, dict):
            entry["type"] = "dict"
            keys = list(res.keys())[:5]
            summary = {k: res[k] for k in keys}
            entry["summary"] = summary
            print(f"Dict keys ({len(res)} total), sample:", json.dumps(summary, indent=2))

        else:
            entry["type"] = "other"
            entry["result"] = str(res)
            print("Result:", res)

    except Exception as e:
        entry["status"] = "error"
        entry["error"] = repr(e)
        print("Error:", repr(e))

    log.append(entry)

# 2) Define test symbol
symbol = "IBM"

# 3) Test free endpoints
safe_call("get_company_overview", client.get_company_overview, symbol)
safe_call("get_income_statement", client.get_income_statement, symbol)
safe_call("get_balance_sheet", client.get_balance_sheet, symbol)
safe_call("get_cash_flow", client.get_cash_flow, symbol)

# 4) Test premium endpoint (NEWS_SENTIMENT)
safe_call("get_news_sentiment", client.get_news_sentiment, symbol, None, None, None, 5)

# 5) Test key-metrics helper
safe_call("calculate_key_metrics", client.calculate_key_metrics, symbol)

print("\nAll tests complete.")

# Optional: Save log to JSON file
with open("api_test_log.json", "w") as f:
    json.dump(log, f, indent=2)



--- get_company_overview ---
Dict keys (52 total), sample: {
  "Symbol": "IBM",
  "AssetType": "Common Stock",
  "Name": "International Business Machines",
  "Description": "International Business Machines Corporation (IBM) is an American multinational technology company headquartered in Armonk, New York, with operations in over 170 countries. The company began in 1911, founded in Endicott, New York, as the Computing-Tabulating-Recording Company (CTR) and was renamed International Business Machines in 1924. IBM is incorporated in New York. IBM produces and sells computer hardware, middleware and software, and provides hosting and consulting services in areas ranging from mainframe computers to nanotechnology. IBM is also a major research organization, holding the record for most annual U.S. patents generated by a business (as of 2020) for 28 consecutive years. Inventions by IBM include the automated teller machine (ATM), the floppy disk, the hard disk drive, the magnetic stripe card, 

## Fetching Daily Stock Data

Get daily adjusted price data for an Indian stock (using NSE/BSE extensions)

In [4]:
# Fetch daily adjusted data for Reliance Industries from NSE
reliance_daily = client.get_daily_adjusted("RELIANCE.NS")

# Display the last 5 data points
reliance_daily.tail()

""


In [8]:
# Plot the closing price
plt.figure(figsize=(12, 6))
plt.plot(reliance_daily.index, reliance_daily['adjusted close'])
plt.title('Reliance Industries - Adjusted Close Price')
plt.xlabel('Date')
plt.ylabel('Price (INR)')
plt.grid(True)
plt.show()

KeyError: 'adjusted close'

<Figure size 1200x600 with 0 Axes>

## Fetching Fundamental Data

Retrieve company overview and financial statements

In [9]:
# Get company overview
overview = client.get_company_overview("RELIANCE.NS")

# Display selected information
selected_info = {
    key: overview.get(key) for key in [
        "Symbol", "Name", "Description", "Sector", "Industry", "MarketCapitalization",
        "PERatio", "PEGRatio", "BookValue", "DividendYield", "EPS", "ProfitMargin", "ROE"
    ]
}

pd.Series(selected_info)

Symbol                  None
Name                    None
Description             None
Sector                  None
Industry                None
MarketCapitalization    None
PERatio                 None
PEGRatio                None
BookValue               None
DividendYield           None
EPS                     None
ProfitMargin            None
ROE                     None
dtype: object

In [ ]:
# Get income statement
income_statement = client.get_income_statement("RELIANCE.NS")

# Extract annual reports and create a DataFrame
annual_reports = income_statement.get("annualReports", [])
if annual_reports:
    # Create a DataFrame with key metrics
    metrics = [
        "fiscalDateEnding", "totalRevenue", "grossProfit", "operatingIncome", 
        "netIncome", "ebitda"
    ]
    
    # Extract data for each year
    data = []
    for report in annual_reports:
        data.append({metric: report.get(metric) for metric in metrics})
        
    # Create and display DataFrame
    income_df = pd.DataFrame(data)
    income_df.set_index("fiscalDateEnding", inplace=True)
    
    # Convert to numeric
    income_df = income_df.apply(pd.to_numeric)
    
    # Display in millions
    (income_df / 1_000_000).round(2)

## Calculate Key Financial Metrics

Use the helper method to calculate key financial metrics

In [ ]:
# Calculate key metrics for Reliance
metrics = client.calculate_key_metrics("RELIANCE.NS")

# Display metrics
pd.Series(metrics)

## News & Sentiment Analysis

Retrieve recent news and sentiment data

In [ ]:
# Get news for Reliance, limit to 10 items
news = client.get_news_sentiment(symbol="RELIANCE.NS", limit=10)

# Process and display news items
news_items = news.get("feed", [])
if news_items:
    # Create a list of news with relevant information
    processed_news = []
    for item in news_items:
        processed_news.append({
            "title": item.get("title"),
            "time_published": item.get("time_published"),
            "sentiment": item.get("overall_sentiment_label"),
            "sentiment_score": item.get("overall_sentiment_score"),
            "url": item.get("url")
        })
    
    # Create and display DataFrame
    news_df = pd.DataFrame(processed_news)
    news_df

## Topic-based News

Retrieve news based on specific topics relevant to Indian markets

In [ ]:
# Get news for specific topics (e.g., economy, ipo, technology)
topic_news = client.get_news_sentiment(topics="economy,ipo,technology", limit=10)

# Process and display topic-based news
topic_items = topic_news.get("feed", [])
if topic_items:
    # Create a list of news with relevant information
    processed_topics = []
    for item in topic_items:
        processed_topics.append({
            "title": item.get("title"),
            "time_published": item.get("time_published"),
            "topics": ", ".join([topic.get("topic") for topic in item.get("topics", [])]),
            "sentiment": item.get("overall_sentiment_label"),
            "url": item.get("url")
        })
    
    # Create and display DataFrame
    topics_df = pd.DataFrame(processed_topics)
    topics_df